In [ ]:
import asyncio
import inspect
import os
import sys
import time
from pprint import pprint
import bookutils
import ipywidgets as widgets

In [ ]:
from bookutils import input,next_inputs
from pythonfile import py_fun1,py_fun2
import re




In [ ]:
class State:
    """Represents the execution at a given point"""
    def __init__(self,event,abs_lineno,code,changed_vars,frame,rel_lineno,fun_name,file,fun_code):
        self.event = event
        self.abs_lineno = abs_lineno
        self.rel_lineno = rel_lineno
        self.code = code
        self.changed_vars = changed_vars
        self.frame = frame #Remove?
        self.fun_name = fun_name
        self.file = file
        self.fun_code = fun_code
        self.parent=None

    def __str__(self):
        return 'Line: ' + str(self.abs_lineno) + ' ('+ str(self.rel_lineno)+'), ('+repr(self.event)+") " + \
               repr(self.code) + '   '+ repr(self.changed_vars) + '   ' + repr(self.fun_name) + '  ('+ repr(self.file)+')'
        # return 'Line: ' + str(self.abs_lineno) + ' ('+ str(self.rel_lineno)+'), ('+repr(self.event)+") " + \
        #        repr(self.fun_code[self.rel_lineno-1]) + '   '+ repr(self.changed_vars) + '   ' + repr(self.fun_name) + '  ('+ repr(self.file)+')'

In [ ]:
class Breakpoint:
    """Represents a breakpoint"""
    def __init__(self,type,line=None,function=None,condition=None,file=None,is_active=True):
        self.type=type
        self.line_number=line
        self.func_name=function
        self.cond_expression = condition
        self.file = file
        self.is_active=is_active
        self.alias = ""

    def break_here(self,step,log,variables,backwards=False):
        """Returns whether this breakpoints hits with the given arguments. The 'backwards' argument indicates if the
        execution happens backwards"""
        if not self.is_active:
            return False

        current_state = log[step]
        if self.type == 'line':
            return current_state.abs_lineno==self.line_number and current_state.file==self.file
        elif self.type == 'func':
            if backwards:
                if step < len(log)-1:
                    prev = log[step+1]
                    return prev.event=="return" and prev.fun_name==self.func_name
            else:
                if step > 1:
                    prev = log[step-1]
                    return prev.event=="call" and prev.fun_name==self.func_name
        elif self.type =='cond':
            if current_state.abs_lineno==self.line_number:
                try:
                    return eval(self.cond_expression,globals(),variables)
                except Exception:
                    return False

        return False

    def is_active(self):
        return self.is_active

    def enable(self):
        self.is_active=True

    def disable(self):
        self.is_active=False

    def set_alias(self,alias):
        self.alias = alias

    def __str__(self):
        if self.type == 'line':
            return f"line  {repr(self.file)}:{self.line_number}  {self.is_active}"
        elif self.type == 'func':
            return f"func  {repr(self.file)}:{self.func_name}  {self.is_active}"
        elif self.type =='cond':
            return f"cond  {repr(self.file)}:{self.line_number}  {self.is_active}  {repr(self.cond_expression)}"
        else:
            return ""

In [ ]:
class UITracer(object):
    """Trace a block of code and create a log for it. Then call a Debugger object which controls the interactive session """
    def __init__(self, interactive_session=True,file=sys.stdout):
        self.original_trace_function = None
        self.file = file
        self.loglist =[]
        self.last_vars = {}
        self.interactive_session=interactive_session

    def changed_vars(self, new_vars):
        changed = {}
        for var_name in new_vars:
            if (var_name not in self.last_vars or
                    self.last_vars[var_name] != new_vars[var_name]):
                changed[var_name] = new_vars[var_name]
        self.last_vars = new_vars.copy()
        return changed

    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        current_state = State(objects[0], objects[1],objects[2],objects[3],objects[4],objects[5],objects[6],objects[7],objects[8])
        self.loglist.append(current_state)


    def traceit(self, frame, event, arg):
        """Tracing function."""
        self.log_state(frame, event, arg)

    def log_state(self, frame, event, arg):

        changes = self.changed_vars(frame.f_locals)
        module = inspect.getmodule(frame.f_code)

        source,function_start = inspect.getsourcelines(frame.f_code)
        path=inspect.getsourcefile(frame)
        _, file = os.path.split(path)
        relative_lineno = frame.f_lineno-function_start + 1
        current_line = source[relative_lineno - 1]
        func_code = source

        if event == 'call':
            self.log(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals.copy(), frame, relative_lineno,frame.f_code.co_name,file,func_code)

        if event == 'line':

            self.log(event, frame.f_lineno, current_line, frame.f_locals.copy(), frame, relative_lineno,frame.f_code.co_name,file,func_code)

        if event == 'return':
            self.log(event, frame.f_lineno, repr(arg), frame.f_locals.copy(), frame, relative_lineno,frame.f_code.co_name,file,func_code)
            self.last_vars = {}  # Delete 'last' variables


    def _traceit(self, frame, event, arg):
        """Internal tracing function."""
        if frame.f_code.co_name == '__exit__':
            # Do not trace our own _exit_() method
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)
        return self

    def __exit__(self, tp, value, traceback):
        """Called at end of `with` block. Turn tracing off. Then start interactive session."""
        sys.settrace(self.original_trace_function)


    def print_log(self):
        for log in self.loglist:
            print(log)

    def get_log(self):
        return self.loglist

In [ ]:
class TimeTravelDebugger(object):
    """Trace a block of code and create a log for it. Then call a Debugger object which controls the interactive session """
    def __init__(self, interactive_session=True,file=sys.stdout):
        self.original_trace_function = None
        self.file = file
        self.loglist =[]
        self.last_vars = {}
        self.interactive_session=interactive_session

    def changed_vars(self, new_vars):
        changed = {}
        variables = new_vars.copy()
        for var in new_vars:
            if var=="self":
                try:
                    val =eval(var,globals(),variables)
                    if callable(type(val)):
                        member_vars = val.__dict__.copy()
                        for member_var,val in member_vars.items():
                            variables["member:"+member_var]=val
                        #variables.update(val.__dict__.copy())
                except Exception:
                    pass

        for var_name in variables:
            if (var_name not in self.last_vars or
                    self.last_vars[var_name] != variables[var_name]):
                changed[var_name] = variables[var_name]
        self.last_vars = variables.copy()
        return changed

    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        current_state = State(objects[0], objects[1],objects[2],objects[3],objects[4],objects[5],objects[6],objects[7],objects[8])
        self.loglist.append(current_state)


    def traceit(self, frame, event, arg):
        """Tracing function."""
        self.log_state(frame, event, arg)

    def log_state(self, frame, event, arg):

        changes = self.changed_vars(frame.f_locals)

        source,function_start = inspect.getsourcelines(frame.f_code)
        path=inspect.getsourcefile(frame)
        _, file = os.path.split(path)
        relative_lineno = frame.f_lineno-function_start + 1
        current_line = source[relative_lineno - 1]
        func_code = source

        if event == 'call':
            self.log(event, frame.f_lineno, frame.f_code.co_name, changes, frame, relative_lineno,frame.f_code.co_name,file,func_code)

        if event == 'line':

            self.log(event, frame.f_lineno, current_line, changes, frame, relative_lineno,frame.f_code.co_name,file,func_code)

        if event == 'return':
            self.log(event, frame.f_lineno, repr(arg), changes, frame, relative_lineno,frame.f_code.co_name,file,func_code)
            self.last_vars = {}  # Delete 'last' variables


    def _traceit(self, frame, event, arg):
        """Internal tracing function."""
        if frame.f_code.co_name == '__exit__':
            # Do not trace our own _exit_() method
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)
        return self

    def __exit__(self, tp, value, traceback):
        """Called at end of `with` block. Turn tracing off. Then start interactive session."""
        sys.settrace(self.original_trace_function)
        if self.interactive_session:
            debugger = Debugger(self.loglist)
            #self.print_log()
            debugger.start()


    def print_log(self):
        for log in self.loglist:
            print(log)

    def get_log(self):
        return self.loglist

In [ ]:
class Debugger(object):
    """This is the core of the interactive debugger. Handles all commands."""
    def __init__(self,execution_log,interactive_session=True):

        self.current_state = None
        self.callstack = None
        self.callstack_depth = -1
        self.current_step = 0
        self.breakpoints = dict()
        self.watchpoints = dict()
        self.interact = interactive_session
        self.variables = {}
        self.interactive_session = interactive_session
        self.execution_log=execution_log


    # def __enter__(self):
    #     with Tracer() as t:
    #         py_fun1(5)
    #         #foo()
    #     self.execution_log = t.get_log()
    #     t.print_log()
    #     print("\n")
    #     print("\n")
    #     print("\n")
    #     if len(self.execution_log)<1:
    #         print("Log is empty")
    #         return
    #     self.construct_callstack()
    #     self.current_state=self.execution_log[self.current_step]
    #     self.step_command()
    #     if self.interactive_session:
    #         self.interaction_loop()

    def start(self):
        """Initialize values and start the interaction."""
        if len(self.execution_log)<1:
            print("Log is empty")
            return
        self.current_state=self.execution_log[0]
        self.construct_callstack()
        if self.interact:
            self.step_command()
            self.interaction_loop()


    # def __exit__(self, exc_type, exc_val, exc_tb):
    #     pass

    def interaction_loop(self):
        """Takes command inputs."""
        while self.interact:
            command = input("(debugger) ")
            self.execute(command)

    def commands(self):
        cmds = [method.replace('_command', '')
                for method in dir(self.__class__)
                if method.endswith('_command')]
        cmds.sort()
        return cmds


    def command_method(self, command):
        if command.startswith('#'):
            return None  # Comment

        possible_cmds = [possible_cmd for possible_cmd in self.commands()
                         if possible_cmd==command]
        if len(possible_cmds) != 1:
            self.help_command(command)
            return None

        cmd = possible_cmds[0]
        return getattr(self, cmd + '_command')

    def execute(self, command):
        sep = command.find(' ')
        if sep > 0:
            cmd = command[:sep].strip()
            arg = command[sep + 1:].strip()
        else:
            cmd = command.strip()
            arg = ""

        method = self.command_method(cmd)
        if method:
            method(arg)


    def step(self):
        """Advances the execution by one step and calculates the variables. Returns false if at the end"""
        if self.current_step<len(self.execution_log)-1:
            self.current_step += 1
            self.current_state = self.execution_log[self.current_step]
            self.construct_vars()
            return True
        else:
            return False

    def step_to_line(self):
        """Advances the execution to the next 'line' event"""
        self.callstack=None
        self.callstack_depth=-1
        init_step = self.current_step
        while self.current_step<len(self.execution_log)-1:
            self.current_step += 1
            self.current_state = self.execution_log[self.current_step]
            if self.current_state.event=="line":
                self.variables=self.construct_vars_at(self.current_step)
                return True
        self.current_step=init_step
        self.current_state=self.execution_log[self.current_step]
        return False

    def backstep_to_line(self):
        """Steps backwards in the execution to the previous 'line' event"""
        init_step = self.current_step
        while self.current_step>1:
            self.current_step -= 1
            self.current_state = self.execution_log[self.current_step]
            if self.current_state.event=="line":
                self.variables=self.construct_vars_at(self.current_step)
                return True
        self.current_step=init_step
        self.current_state=self.execution_log[self.current_step]
        return False


    def step_back(self):
        """Steps backwards in the execution"""
        if self.current_step> 0:
            self.current_step -= 1
            self.current_state = self.execution_log[self.current_step]
            return True
        else:
            return False


    def construct_callstack(self):
        current_parent = None
        callpoints = list()
        current_depth = 0
        for i in range(0,len(self.execution_log)):
            currentexecution = self.execution_log[i]
            currentexecution.parent = current_parent

            if currentexecution.event == 'call':
                current_parent = self.execution_log[i]
                if self.execution_log[i].parent is not None :
                    current_caller = self.execution_log[i-1]
                    callpoints.append(current_caller)
                    current_parent = current_caller
                else:
                    callpoints.append(current_parent)

                current_depth = current_depth + 1

            elif currentexecution.event == 'return':
                current_depth = current_depth - 1
                caller = callpoints[current_depth]
                current_parent = caller.parent


    def construct_current_callstack(self):
        current_callstack = list()
        current_caller = self.current_state

        while current_caller is not None :
            if current_caller.parent!=current_caller:
                current_callstack.append(current_caller)
                current_caller = current_caller.parent

        current_callstack.reverse()
        return current_callstack


    def where_command(self,arg=""):
        """Print the current call stack"""
        current_callstack = self.construct_current_callstack()
        if self.callstack_depth == -1:
            self.callstack_depth = len(current_callstack)-1

        for i in range(0,len(current_callstack)):
            call = current_callstack[i]
            if i == self.callstack_depth:
                #self.log('>> File "'+ str(self.current_state.file)+'", Line '+ str(self.current_state.abs_lineno)  + ', in '+str(self.current_state.fun_name))
                self.log('>> File "'+ str(call.file)+'", Line '+ str(call.abs_lineno)  + ', in '+str(call.fun_name))
            else:
                if call:
                    if call.parent:
                        self.log('File "'+str(call.file)+'", Line '+ str(call.abs_lineno)  + ', in '+ str(call.fun_name))
                    else:
                        self.log('File "'+str(call.file)+'", Line '+ str(call.abs_lineno)  + ', in '+ "<module>")


    def get_execution_index(self, execution):
        for i in range(0,len(self.execution_log)):
            if execution==self.execution_log[i]:
                return i


    def up_command(self,arg=""):
        """Move up the call stack towards the callers"""
        current_callstack = self.construct_current_callstack()
        if self.callstack_depth==-1:
            self.callstack_depth=len(current_callstack)-1
        if self.callstack_depth>1:
            self.callstack_depth -= 1
        else:
            self.log("at the top already")
            return


        call = current_callstack[self.callstack_depth]
        # for line in call.fun_code:
        #     self.log(line)

        for i in range(0,len(call.fun_code)):
            if call.rel_lineno-1==i:
                self.log(">>"+call.fun_code[i])
            else:
                self.log(call.fun_code[i])


    def down_command(self,arg=""):
        """Move down the call stack towards the callees"""
        current_callstack=self.construct_current_callstack()
        if self.callstack_depth==-1:
            self.callstack_depth=len(current_callstack)-1
        if self.callstack_depth==len(current_callstack)-1:
            self.log("Already on lowest layer of call stack")
        else:
            self.callstack_depth+=1
            call = current_callstack[self.callstack_depth]
            # for line in call.fun_code:
            #     self.log(line)
                
            for i in range(0,len(call.fun_code)):
                if call.rel_lineno-1==i:
                    self.log(">>"+call.fun_code[i])
                else:
                    self.log(call.fun_code[i])



    def backconstruct_vars(self):
        """Construct the variables for the current step"""
        backconstructed_vars = dict()
        if self.current_step>0:
            num_subroutines = 0
            for i in range(0,self.current_step):
                current_execution = self.execution_log[self.current_step-(i+1)]
                if current_execution.event == 'return':
                    num_subroutines  +=1
                elif current_execution.event == 'call':
                    if num_subroutines == 0: break
                    num_subroutines -=1
                elif num_subroutines==0:
                    changed_vars = current_execution.changed_vars.copy()
                    changed_vars.update(backconstructed_vars)
                    backconstructed_vars = changed_vars

            self.variables = backconstructed_vars
            return  True
        else:
            self.variables=self.execution_log[0].changed_vars.copy()
            return False

    def construct_vars_at(self,target_step):
        """Constructs the variables at any execution step"""
        constructed_vars = dict()
        if target_step>0:
            num_subroutines = 0
            for i in range(0,target_step+1):
                current_execution = self.execution_log[target_step-i]
                if current_execution.event == 'return':
                    if i>0:
                        num_subroutines  +=1
                elif current_execution.event == 'call':
                    if num_subroutines == 0:
                        changed_vars = current_execution.changed_vars.copy()
                        changed_vars.update(constructed_vars)
                        constructed_vars = changed_vars
                        break
                    num_subroutines -=1
                elif num_subroutines==0:
                    changed_vars = current_execution.changed_vars.copy()
                    changed_vars.update(constructed_vars)
                    constructed_vars = changed_vars
        else:
            self.variables=self.execution_log[0].changed_vars.copy()
        return constructed_vars

    def construct_vars(self):
        """Constructs variables at the current step when advancing one step"""
        event = self.current_state.event
        if event == 'return':
            self.variables.clear()
        elif event == 'call':
            self.variables = self.current_state.changed_vars.copy()
        else:
            self.variables.update(self.current_state.changed_vars)


    def next_command(self,arg=""):
        """Step over function calls going to the next line"""
        if self.current_step<len(self.execution_log)-1:
            if self.execution_log[self.current_step+1].event=="call":
                self.step()
                fun_name = self.current_state.fun_name
                depth = self.call_depth()
                while not (self.current_state.event == 'return' and self.current_state.fun_name == fun_name and self.call_depth()-1 ==depth):
                    self.step()
                    self.variables = self.construct_vars_at(self.current_step)
                    if self.has_breakpoint():
                        self.check_watchpoints()
                        self.print_debugger_status()
                        return

        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def previous_command(self,arg=""):
        """Step over function calls going to the previous line"""
        if self.current_step>1:
            fun_name = self.current_state.fun_name
            depth = self.call_depth()
            if self.execution_log[self.current_step-1].event=="return":
                self.backstep_to_line()
                while not (self.current_state.fun_name == fun_name and self.call_depth()==depth):
                    if self.has_breakpoint(True):
                        break
                    self.backstep_to_line()
            else:
                self.backstep_to_line()

        self.check_watchpoints()
        self.print_debugger_status()

    def call_depth(self,state=None):
        """Returns the depth of the current (or given step)"""
        if not state:
            state = self.current_state
        depth = 0
        while state.parent:
            state = state.parent
            depth += 1
        return depth

    def next_state(self):
        if self.current_step<len(self.execution_log)-1:
            return self.execution_log[self.current_step+1]
        else:
            return None

    def lines_above(self,num_lines):
        """Returns num_lines lines of code of the current function before the current step"""
        surrounding_lines = list()
        fun_code = self.current_state.fun_code
        if fun_code:
            index = self.current_state.rel_lineno -1

            for i in range(max(0,index-num_lines),index):
                surrounding_lines.append(fun_code[i])
        return surrounding_lines


    def lines_below(self,num_lines):
        """Returns num_lines lines of code of the current function after the current step"""
        surrounding_lines = list()
        fun_code = self.current_state.fun_code
        if fun_code:
            index = self.current_state.rel_lineno - 1
            if index<len(fun_code)-1:
                for i in range(index+1, min(len(fun_code),index+num_lines+1)):
                    surrounding_lines.append(fun_code[i])
        return surrounding_lines


    def list_command(self,args=""):
        """Print the source code around the current line
        'list' prints two lines above and below
        'list <number>' prints <number> lines above and below
        'list <above> <below>' prints <above> lines above and <below> lines below"""
        arg_iterator = map(int, re.findall(r'\d+', args))

        if args!="" and re.match('^[0-9 |' ']*$',args):

             parsed_args = list()

             for parsed_arg in arg_iterator:
                 parsed_args.append(parsed_arg)
             surrounding_lines = list()

             if len(parsed_args) == 0:

                 surrounding_lines = self.lines_above(2)
                 surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> "+ str(self.current_state.code))]
                 surrounding_lines = surrounding_lines + self.lines_below(2)
                 for surrounding_line in surrounding_lines:
                      print(str(surrounding_line))

             elif len(parsed_args) == 1:
                 surrounding_lines = surrounding_lines + (self.lines_above(parsed_args[0]))
                 surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> " + str(self.current_state.code))]
                 surrounding_lines = surrounding_lines + (self.lines_below(parsed_args[0]))
                 for surrounding_line in surrounding_lines:
                      print(str(surrounding_line))

             elif len(parsed_args) == 2:
                 surrounding_lines  = surrounding_lines + (self.lines_above(parsed_args[0]))
                 surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> " + str(self.current_state.code))]
                 surrounding_lines  = surrounding_lines + (self.lines_below(parsed_args[1]))
                 for surrounding_line in surrounding_lines:
                      print(str(surrounding_line))
             else:
                self.log("Wrong usage: list <above> <below> :Print <above> lines before and <below> lines after the current line")

        elif args.replace(" ","")== "":
             surrounding_lines = self.lines_above(2)
             surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> "+ str(self.current_state.code))]
             surrounding_lines = surrounding_lines + self.lines_below(2)
             for surrounding_line in surrounding_lines:
                  print(str(surrounding_line))
        else:
            self.log("Wrong usage: list <above> <below> :Print <above> lines before and <below> lines after the current line")


    def until_command(self,arg=""):
        """Execute forwards until a certain point.
        'until' executes until a line greater than the current line is reached
        'until <line_number>' executes until a line greater than <line_number> is reached
        'until <filename>:<line_number>' executes until line <line_number> is reached in file <filename>
        'until <function_name>' executes until function <function_name> is called
        'until <filename>:<function_name>' executes until function <function_name> declared in <filename> is called
        """
        #TODO special case no arg act as next
        if not arg:
            #no arg case
            self.until_line(self.current_state.abs_lineno,self.current_state.file)
            return

        function_name=""
        target_line=0
        sep = arg.find(':')
        if sep > 0:
            #compound arg case
            filename = arg[:sep].strip()
            secondary_arg = arg[sep+1:].strip()
            if secondary_arg.isdigit():
                target_line=int(secondary_arg)
            else: 
                function_name = secondary_arg

            if target_line > 0:
                self.until_line(target_line,filename)
            else:
                self.until_function(function_name,filename)
        else:
            #simple arg case
            if arg.isdigit():
                target_line=int(arg)
            else:
                function_name=arg

            if function_name:
                self.until_function(function_name,self.current_state.file)
            else:
                self.until_line(target_line,self.current_state.file)


    def backuntil_command(self,arg=""):
        """Execute backwards until a certain point.
        'backuntil' executes backwards until a line less than the current line is reached
        'backuntil <line_number>' executes backwards until a line less than <line_number> is reached
        'backuntil <filename>:<line_number>' executes backwards until line <line_number> is reached in file <filename>
        'backuntil <function_name>' executes backwards until function <function_name> is called
        'backuntil <filename>:<function_name>' executes backwards until function <function_name> declared in <filename> is called
        """
        if not arg:
            #no arg case
            self.backuntil_line(self.current_state.abs_lineno,self.current_state.file)
            return

        function_name=""
        target_line=0
        sep = arg.find(':')
        if sep > 0:
            #compound arg case
            filename = arg[:sep].strip()
            secondary_arg = arg[sep+1:].strip()
            if secondary_arg.isdigit():
                target_line=int(secondary_arg)
            else:
                function_name = secondary_arg
            if target_line > 0:
                self.backuntil_line(target_line,filename)
            else:
                self.backuntil_function(function_name,filename)
        else:
            #simple arg case
            if arg.isdigit():
                target_line=int(arg)
            else:
                function_name=arg

            if function_name:
                self.backuntil_function(function_name,self.current_state.file)
            else:
                self.backuntil_line(target_line,self.current_state.file)

    def until_function(self,target_function,target_file):
        """Advance until call to target_function and in target_file if given"""
        stepped = True
        while not (self.current_state.fun_name == target_function and self.current_state.event=="call"
                   and self.current_state.file==target_file ):
            stepped = self.step()
            self.variables=self.construct_vars_at(self.current_step)
            if not stepped:
                break
            if self.has_breakpoint():
                self.check_watchpoints()
                self.print_debugger_status()
                return

        if stepped:
            self.step_to_line()
        else:
            self.backstep_to_line()
        self.check_watchpoints()
        self.print_debugger_status()


    def backuntil_function(self,target_function,target_file):
        """Run backwards until call to target_function and in target_file if given"""
        while not (self.current_state.fun_name == target_function and self.current_state.event=="call" and
                    (self.current_state.file==target_file)):
            stepped = self.step_back()
            if not stepped:
                break

            self.variables=self.construct_vars_at(self.current_step)
            if self.has_breakpoint(True):
                self.check_watchpoints()
                self.print_debugger_status()
                return

        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()


    def until_line(self,target_line,target_file):
        """Advance until program line is greater than target_line and in target_file if given"""
        if target_line<=0:
            self.log("Invalid line: " + str(target_line))
            return

        for i in range(self.current_step,len(self.execution_log)):
            if self.current_state.abs_lineno>target_line and self.current_state.file==target_file:
                break
            self.step_to_line()
            if self.has_breakpoint():
                break

        self.check_watchpoints()
        self.print_debugger_status()


    def backuntil_line(self,target_line,target_file):
        """Run backwards until program line is lower than target_line and in target_file"""
        if target_line<=1:
            self.log("Invalid line: " + str(target_line))
            return

        for i in range(0,self.current_step+1):
            if self.current_state.abs_lineno<target_line and self.current_state.file==target_file:
                break
            self.backstep_to_line()
            if self.has_breakpoint(True):
                break

        self.check_watchpoints()
        self.print_debugger_status()


    def finish_command(self,arg=""):
        """Execute until return of current function"""
        fun_name = self.current_state.fun_name
        depth = self.call_depth()
        while not (self.current_state.fun_name == fun_name and self.current_state.event=="return" and self.call_depth()==depth):
            self.step()
            self.variables = self.construct_vars_at(self.current_step)
            if self.has_breakpoint():
                self.check_watchpoints()
                self.print_debugger_status()
                return

        self.backstep_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def start_command(self,arg=""):
        """Execute backwards until a function start"""
        fun_name = self.current_state.fun_name
        depth = self.call_depth()
        while not (self.current_state.fun_name == fun_name and self.current_state.event=="call" and self.call_depth()+1==depth):
            self.step_back()
            self.variables = self.construct_vars_at(self.current_step)
            if self.has_breakpoint(True):
                self.check_watchpoints()
                self.print_debugger_status()
                return
        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def has_breakpoint(self,backwards=False):
        """Checks if any of the breakpoints trigger"""
        active_breakpoints = [bp for bp in self.breakpoints if self.breakpoints[bp].break_here(self.current_step,self.execution_log,self.variables,backwards)]
        return len(active_breakpoints)>0

    def quit_command(self,arg=""):
        """End the debugger session"""
        self.interact=False
        self.log("Quit debugger session")

    def continue_command(self, arg=""):
        """ Continue execution forward until a breakpoint is hit, or the program finishes"""
        stepping = True
        while stepping:
            stepped = self.step_to_line()
            if (not stepped) or self.has_breakpoint():
                stepping = False

        self.check_watchpoints()
        self.print_debugger_status()

    def reverse_command(self, arg=""):
        """Continue execution backward until a breakpoint is hit, or the program starts"""
        stepping = True
        while stepping:
            stepped = self.backstep_to_line()
            if not stepped or self.has_breakpoint(True):
                stepping = False
        self.check_watchpoints()
        self.print_debugger_status()

    def backstep_command(self,arg=""):
        """Step to the previous executed line"""
        self.backstep_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def step_command(self, arg=""):
        """Execute up to the next line"""
        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def print_command(self, arg=""):
        """Print an expression. If no expression is given, print all variables"""
        #vars = self.current_state.frame.f_locals
        vars = self.variables
        if not arg:
            self.log("\n".join([f"{var} = {repr(vars[var])}" for var in vars]))
            #self.log("\n".join([f"{var} = {repr(self.current_state.frame.f_globals[var])}" for var in self.current_state.frame.f_globals]))
        else:
            try:
                self.log(f"{arg} = {repr(eval(arg, globals(), vars))}")
            except Exception as err:
                self.log(f"{err.__class__.__name__}: {err}")

    def break_command(self, arg=""):
        """Set a breakpoint.
        'break <line_number>' Set breakpoint in line <line_number>
        'break <function_name>' Set a breakpoint which hits when a function with the name <function_name> is entered
        'break <file_name>:<function_name>' Set a breakpoint which hits when a function <function_name> declared in <file_name> is entered"""
        if arg:
            if len(self.breakpoints)>0:
                largest_key= max(self.breakpoints.keys())
            else:
                largest_key = 0

            if arg.isdigit():
                self.breakpoints[largest_key+1]=Breakpoint("line",int(arg),None,None,self.current_state.file)
            else:
                sep = arg.find(':')
                if sep>0:
                    file_name = arg[:sep].strip()
                    func_name = arg[sep+1:].strip()
                    self.breakpoints[largest_key+1]=Breakpoint("func",None,func_name,None,file_name)
                else:
                    self.breakpoints[largest_key+1]=Breakpoint("func",None,arg,None,self.current_state.file)
        else:
            self.log("Missing argument, use as 'break <line>|<function_name>|<file_name>:<function_name>'")

    def delete_command(self, arg=""):
        """Delete breakpoint with given <breakpoint_id>|<breakpoint_name>"""
        if arg:
            breakpoint_id = self.breakpoint_lookup(arg)
            if breakpoint_id==-1:
                self.log("No such breakpoint: "+arg)
            else:
                self.breakpoints.pop(breakpoint_id)
        else:
            self.log("Missing argument, use as 'delete <breakpoint_id>'")

    def breakpoints_command(self,arg=""):
        """Display all available breakpoints"""
        for breakpoint in self.breakpoints:
            alias = self.breakpoints[breakpoint].alias
            if alias:
                self.log(alias+ "   " +str(self.breakpoints[breakpoint]))
            else:
                self.log(str(breakpoint)+ "   " +str(self.breakpoints[breakpoint]))

    def disable_command(self,arg=""):
        """Use as 'disable <breakpoint_id>|<breakpoint_name>'    Suspend  specified breakpoint"""
        if arg:
            breakpoint_id = self.breakpoint_lookup(arg)
            if breakpoint_id==-1:
                self.log("No such breakpoint: "+arg)
            else:
                self.breakpoints[breakpoint_id].disable()
        else:
            self.log("Missing argument, use as 'disable <breakpoint_id>'")

    def enable_command(self,arg=""):
        """Use as 'enable <breakpoint_id>|<breakpoint_name>'    Re-enable specified breakpoint"""
        if arg:
            breakpoint_id = self.breakpoint_lookup(arg)
            if breakpoint_id==-1:
                self.log("No such breakpoint: "+arg)
            else:
                self.breakpoints[breakpoint_id].enable()
        else:
            self.log("Missing argument, use as 'enable <breakpoint_id>'")

    def breakpoint_lookup(self,arg):
        """Find key of given breakpoint_id or breakpoint_name. Returns -1 if no such breakpoint exists"""
        if arg.isdigit():
            if int(arg) in self.breakpoints.keys():
                return int(arg)

        else:
            for breakpoint_id in self.breakpoints:
                if self.breakpoints[breakpoint_id].alias==arg:
                    return breakpoint_id
        return -1

    def alias_command(self,arg=""):
        """Use as 'alias <breakpoint_id> <breakpoint_name>'  Set the alias <breakpoint_name> for breakpoint <breakpoint_id> """
        if not arg:
            self.log("Requires arguments <breakpoint_id> <breakpoint_name>")

        sep = arg.find(' ')
        if sep > 0:
            breakpoint_id = arg[:sep].strip()
            alias = arg[sep+1:].strip()
            if not alias:
                self.log("Empty string not allowed as alias")
                return
            if breakpoint_id.isdigit():
                breakpoint_aliases=[bp.alias for bp in self.breakpoints.values()]
                if alias in breakpoint_aliases:
                    self.log("Alias already exists: "+ alias)
                    return
                try:
                    breakpoint = self.breakpoints[int(breakpoint_id)]
                    breakpoint.alias=alias
                except KeyError:
                    self.log("No breakpoint with id: " + breakpoint_id)
            else:
                self.log("Invalid arguments. Use as 'alias <breakpoint_id> <breakpoint_name>'")

        else:
            self.log("Invalid arguments. Use as 'alias <breakpoint_id> <breakpoint_name>'")


    def cond_command(self,arg=""):
        """Use as 'cond <line> <condition>'    Set a breakpoint at which the execution is stopped at line <line>
        if a condition <condition> is true"""
        if arg:
            sep = arg.find(' ')
            if sep>0:
                line = arg[:sep].strip()
                cond_expression = arg[sep+1:].strip()
                if not line.isdigit():
                    self.log("Faulty argument, use as 'cond <line> <condition>'")
                    return

                if len(self.breakpoints)>0:
                    largest_key= max(self.breakpoints.keys())
                else:
                    largest_key = 0

                self.breakpoints[largest_key+1]=Breakpoint("cond",int(line),None,cond_expression,self.current_state.file)
            else:
                self.log("Faulty argument, use as 'cond <line> <condition>'")
        else:
            self.log("Missing argument, use as 'cond <line> <condition>'")

    def watch_command(self,arg=""):
        """Creates a numbered watchpoint for the given variable"""
        if arg:
            variables = [var for (var, val) in self.watchpoints.values()]
            if not arg in variables:
                if len(self.watchpoints)>0:
                    largest_key= max(self.watchpoints.keys())
                else:
                    largest_key = 0
                self.watchpoints[largest_key+1]=(arg,None)

            else:
                self.log("Watchpoint already exists: "+arg)
        else:
            self.print_watchpoints()

    def unwatch_command(self,arg=""):
        """Remove a watchpoint with the given <watch_id>"""
        if arg:
            if arg.isdigit():
                try:
                    self.watchpoints.pop(int(arg))
                except KeyError:
                    self.log("No such watchpoint: "+ arg)
                self.log("Watchpoints", self.watchpoints)
            else:
                self.log("The argument has to be an integer: unwatch <watch_id>")
        else:
            self.log("Specify which watchpoint to remove: unwatch <watch_id>")
            self.log("Watchpoints", self.watchpoints)

    def update_watchpoints(self,print_changes=False):
        """Checks whether any of the watchpoints trigger and prints the corresponding output"""
        changes={}
        for watch_id in self.watchpoints:
            entry = self.watchpoints[watch_id]
            if entry[0] in self.variables:
                if self.variables[entry[0]]!=entry[1]:
                    changes[watch_id]=(entry[0],self.variables[entry[0]])
        self.watchpoints.update(changes)
        if print_changes and changes:
            changes_s= ", ".join([repr(self.watchpoints[watch_id][0])+" = "+ repr(self.watchpoints[watch_id][1])
                               for watch_id in changes])
            self.log("Changed watchpoints: " + changes_s)
        return changes

    def check_watchpoints(self):
        """Controls watchpoint updates. If the program just returned a watchpoint only triggers if that is due to
        being assigned to the return value"""
        if not self.watchpoints:
            return

        if self.current_step>0:
            if self.execution_log[self.current_step-1].event == 'return':
                var_backup = self.variables.copy()
                step = self.current_step-1
                state =self.execution_log[step]
                while not (step == 0 or (state.event=='line' and state.fun_name == self.current_state.fun_name)):
                    step -= 1
                    state = self.execution_log[step]
                if step != 0:
                    self.log(str(state))
                    self.variables = self.construct_vars_at(step)
                    self.update_watchpoints()
                    self.variables=var_backup
                    self.update_watchpoints(True)
            else:
                self.update_watchpoints(True)
        else:
            self.update_watchpoints(True)

    def print_watchpoints(self):
        "Print all current watchpoints"
        watched_vars= ", ".join([str(watch_id) + ": "+repr(self.watchpoints[watch_id][0])
                               for watch_id in self.watchpoints])
        self.log(watched_vars)


    def print_debugger_status(self):
        """Print the currently viewed line of the debugger"""
        event = self.current_state.event

        if event == 'call':
            self.log("This is a call event, which should never be printed")

        if event == 'line':
            current_line = self.current_state.code
            self.log(repr(self.current_state.abs_lineno) + '    ' + current_line)
        if event == 'return':
            self.log("This is a return event, which should never be printed")

    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        print(*objects, sep=sep, end=end, file=sys.stdout, flush=True)

    def help_command(self, command=""):
        """Give help on given command. If no command is given, give help on all"""
        if command:
            possible_cmds = [possible_cmd for possible_cmd in self.commands()
                             if possible_cmd.startswith(command)]

            if len(possible_cmds) == 0:
                print(f"Unknown command {repr(command)}. Possible commands are:")
                possible_cmds = self.commands()
            elif len(possible_cmds) > 1:
                print(f"Ambiguous command {repr(command)}. Possible expansions are:")
        else:
            possible_cmds = self.commands()

        for cmd in possible_cmds:
            method = self.command_method(cmd)
            print(f"{cmd:10} -- {method.__doc__}")

    def get_log(self):
        return self.execution_log

In [ ]:
def foo():
    a = 12
    a= 13
    a = bar(21)
    x = 5
    recursion(3)
    #b = fun(a,x)
    a = 11
    return

def bar(arg):
    b = arg
    a = 7
    return b
    #code()

def code():
    return

def recursion(value):
    if value>0:
        recursion(value - 1)




In [ ]:
with TimeTravelDebugger():
    py_fun1(1)



In [ ]:
#d= Debugger(res)
#d.start()
#next_inputs(["step","print","step","step","print","step","print s","continue"])
#next_inputs(["step","step","step","step","step","step","continue"])
#next_inputs(["step","break 4","continue","continue"])
#with Debugger():
#    remove_html_markup("<b>bold</b>")

In [ ]:
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

outputstream = widgets.Output()
out = widgets.HTML(
                    value='<h3> dasdadsdssad </h3>',
                    placeholder='',
                    description='',
                )
def whichNumber(number):
    global out
    outputstream.append_stdout(str(number))
    out.value = str(number)

display(out)
int_range = widgets.IntSlider(min=1,max=10,value=1,description='Value')
interact(whichNumber,number=int_range)

In [ ]:

with TimeTravelDebugger(False) as ttd:
    foo()
execution_log = ttd.get_log()
debugger = Debugger(execution_log=execution_log,interactive_session=False)
debugger.start()

In [ ]:
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, layout=widgets.Layout(height='auto', width='auto'))

play = widgets.Play(
#     interval=10,
    value=0,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
tester = widgets.HBox([play, slider])


top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

top_left_text = widgets.IntText(description='Top left', layout=widgets.Layout(width='auto', height='auto'))
top_right_text = widgets.IntText(description='Top right', layout=widgets.Layout(width='auto', height='auto'))
bottom_left_slider = widgets.IntSlider(description='Bottom left', layout=widgets.Layout(width='auto', height='auto'))
bottom_right_slider = widgets.IntSlider(description='Bottom right', layout=widgets.Layout(width='auto', height='auto'))

html_code = list()
functions = list()
lines = list()

for execution in execution_log:
    if execution.fun_code not in functions:
        current_function = list()
        for line in execution.fun_code:
            current_function.append(line)
        functions.append(current_function)

for function in functions:
    for line in function:
        if 'def ' in line:
            my_html = widgets.HTML(
                    value='<h3>'+ line + '</h3>',
                    placeholder='',
                    description='',
                )
            html_code.append(my_html)
        else:
            my_html = widgets.HTML(
                    value='<h4>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'+ line + '</h4>',
                    placeholder='',
                    description='',
                )
            html_code.append(my_html)

def whichLine(number):
    global html_code
    html_code[number].value='>> ' + html_code[number.value]

int_range = widgets.IntSlider(min=1,max=len(execution_log),value=1,description='Execution Step')
interact(whichLine,number=int_range)

box = widgets.VBox(html_code)
box.observe(whichLine)

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

widgets.AppLayout(header=tester,
          center=box,
          right_sidebar=right_button,
          pane_widths=[2, 0.5, 1],
          pane_heights=[1, 17, 1])

In [ ]:
with UITracer() as tr:
    foo()

In [ ]:
log = tr.get_log()
#for l in log:
    #print(str(l))
a = widgets.IntSlider(min=0,max=len(log)-1,value=1,description='step')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')

# def state_display(number):
#     log = debugger.get_log()
#     fun_code = log[number].fun_code.copy()
#     fun_code[log[number].rel_lineno-1] = ">>"+fun_code[log[number].rel_lineno-1]
#
#     for line in fun_code:
#         print(line)

def code_display(step):
    #fun_code = "\n".join([line for line in log[number].fun_code])
    fun_code = log[step].fun_code.copy()
    fun_code[log[step].rel_lineno-1] = ">>"+fun_code[log[step].rel_lineno-1]

    for line in fun_code:
        print(line)

def var_display(step):
    print(log[step].changed_vars)
    #print(log[number].changed_vars)

def step(b):
    a.value+=1

def backstep(b):
    a.value -=1

def continue_exec(b):
    while a.value<a.max:
        a.value+=1
        time.sleep(0.1)

def reverse_exec(b):
    while a.value>0:
        a.value-=1
        time.sleep(0.1)

out = widgets.interactive_output(code_display,{'step':a})
out2 = widgets.interactive_output(var_display,{'step':a})

step_btn = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='step-forward'
)

backstep_btn = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='step-backward'
)

contn_btn = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='fast-forward'
)

reverse_btn = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='fast-backward'
)

step_btn.on_click(step)
backstep_btn.on_click(backstep)
contn_btn.on_click(continue_exec)
reverse_btn.on_click(reverse_exec)

widgets.HBox([widgets.VBox([a,contn_btn,reverse_btn,step_btn,backstep_btn]),out, out2])

In [ ]:
log = debugger.get_log()
for l in log:
    print(str(l))
a = widgets.IntSlider(min=0,max=len(log)-1,value=1,description='step')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')

def state_dipaly(number):
    log = debugger.get_log()
    fun_code = log[number].fun_code.copy()
    fun_code[log[number].rel_lineno-1] = ">>"+fun_code[log[number].rel_lineno-1]

    for line in fun_code:
        print(line)

def code_display(number):
    #fun_code = "\n".join([line for line in log[number].fun_code])
    fun_code = log[number].fun_code.copy()
    fun_code[log[number].rel_lineno-1] = ">>"+fun_code[log[number].rel_lineno-1]

    for line in fun_code:
        print(line)

def var_display(number):
    print(debugger.construct_vars_at(number))
    #print(log[number].changed_vars)

out = widgets.interactive_output(code_display,{'number':a})
out2 = widgets.interactive_output(var_display,{'number':a})

widgets.HBox([widgets.VBox([a]), out, out2])